# Feature Importance

In our project, we are predicting gene expression using ChIP-seq profiles. One question of interest is which epigenetics features are the most important to predict gene expression.

Author: Karthik Guruvayurappan

In [27]:
# for computation + data frames
import numpy as np
import pandas as pd

# for file operations
import os

# models
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

## Read in Epigenetic Data (from Eryk)

In [2]:
# data loading code (from Eryk)
master_folder_path = 'Multiome'

dfs_epigenetics = {}
dfs_transcriptomics = {}

# Iterate through all subdirectories and files in the master folder
for root, dirs, files in os.walk(master_folder_path):
    # If we are at the third level of directories get the histone modification assay data
    if len(root.split("/")) == 3:
        cell_line = root.split("/")[1]
        if cell_line not in dfs_epigenetics:
            dfs_epigenetics[cell_line] = []
        for file in files:
            if file.endswith('.csv'):
                # Construct the full path to the CSV file
                csv_file_path = os.path.join(root, file)
                histone_mark = csv_file_path.split("_")[-1].strip(".csv")
        
                # Read the CSV file using pandas
                df = pd.read_csv(csv_file_path)
                df['annotation'] = df['annotation'].apply(lambda x: x.split(" (")[0])
                df['annotation'] = df['annotation'] + f"_{histone_mark}"
                dfs_epigenetics[cell_line].append(df)
    # If we are at the second level of directories get RNA seq data
    if len(root.split("/")) == 2:
        cell_line = root.split("/")[1]
        for file in files:
            if file.endswith('.tsv'):
                tsv_file_path = os.path.join(root, file)
                rna_seq = pd.read_csv(tsv_file_path, sep = '\t')
                dfs_transcriptomics[cell_line] = rna_seq

In [3]:
df_aggregated = pd.concat(dfs_epigenetics['Mammary Epithelial Cell'])

In [4]:
# Get the number of peaks per gene region
peak_count_df = df_aggregated.groupby(["geneId","annotation"],as_index=False).agg(
    peak_counts = ('annotation','size')
)

In [5]:
feature_matrix = peak_count_df.pivot_table(columns='annotation',index=['geneId'])['peak_counts']

In [6]:
feature_matrix.shape

(28763, 77)

In [7]:
feature_matrix.fillna(0, inplace = True)

In [8]:
feature_matrix.head()

annotation,3' UTR_H2AFZ,3' UTR_H3K27a,3' UTR_H3K27me3,3' UTR_H3K36me3,3' UTR_H3K4me1,3' UTR_H3K4me2,3' UTR_H3K4me3,3' UTR_H3K79me2,3' UTR_H3K9a,3' UTR_H3K9me3,...,Promoter_H3K27a,Promoter_H3K27me3,Promoter_H3K36me3,Promoter_H3K4me1,Promoter_H3K4me2,Promoter_H3K4me3,Promoter_H3K79me2,Promoter_H3K9a,Promoter_H3K9me3,Promoter_H4K20me1
geneId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,2.0,2.0,0.0,1.0,2.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


## Read in RNA-seq Data (from Eryk)

In [9]:
rna_seq = dfs_transcriptomics['Mammary Epithelial Cell']

In [10]:
rna_seq.shape

(59526, 17)

In [11]:
rna_seq.head()

,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12958,12958,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12960,12960,73.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
rna_seq['gene_id_clean'] = rna_seq['gene_id'].apply(lambda x: x.split(".")[0])

## Merge datasets by gene name

In [13]:
mapping = pd.read_table('mart_export_grch38.txt')

In [14]:
mapping.head()

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Gene name,NCBI gene (formerly Entrezgene) ID
0,ENSG00000210049,ENSG00000210049.1,ENST00000387314,ENST00000387314.1,MT-TF,NaN
1,ENSG00000211459,ENSG00000211459.2,ENST00000389680,ENST00000389680.2,MT-RNR1,NaN
2,ENSG00000210077,ENSG00000210077.1,ENST00000387342,ENST00000387342.1,MT-TV,NaN
3,ENSG00000210082,ENSG00000210082.2,ENST00000387347,ENST00000387347.2,MT-RNR2,NaN
4,ENSG00000209082,ENSG00000209082.1,ENST00000386347,ENST00000386347.1,MT-TL1,NaN


In [15]:
ensembl_gene_id_map = mapping.set_index('Gene stable ID').to_dict()['Gene name']
ensembl_transcript_id_map = mapping.set_index('Transcript stable ID version').to_dict()['Gene name']

# NCBI column has NaNs which turns the integers into floats, so drop NaNs, change floats --> int --> str, and create dict
ncbi_gene_id_map = mapping.dropna(subset=['NCBI gene (formerly Entrezgene) ID'])
ncbi_gene_id_map['NCBI gene (formerly Entrezgene) ID'] = ncbi_gene_id_map['NCBI gene (formerly Entrezgene) ID'].astype(int).astype(str)
ncbi_gene_id_map = ncbi_gene_id_map.set_index('NCBI gene (formerly Entrezgene) ID').to_dict()['Gene name']

# Combine the transcript IDs, gene IDs, and NCBI IDs dictionaries for all possible mappings
mapping_dict = ensembl_transcript_id_map | ensembl_gene_id_map | ncbi_gene_id_map 

/var/folders/gd/yfplhppn3jx0g03j2lr_yn6m0000gn/T/ipykernel_46973/2742020782.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ncbi_gene_id_map['NCBI gene (formerly Entrezgene) ID'] = ncbi_gene_id_map['NCBI gene (formerly Entrezgene) ID'].astype(int).astype(str)


In [16]:
rna_seq['gene_name'] = rna_seq['gene_id_clean'].map(mapping_dict)
feature_matrix['gene_name'] = feature_matrix.index.astype(str).map(mapping_dict)

## Train a model!

In [19]:
feature_matrix.head()

annotation,3' UTR_H2AFZ,3' UTR_H3K27a,3' UTR_H3K27me3,3' UTR_H3K36me3,3' UTR_H3K4me1,3' UTR_H3K4me2,3' UTR_H3K4me3,3' UTR_H3K79me2,3' UTR_H3K9a,3' UTR_H3K9me3,...,Promoter_H3K27me3,Promoter_H3K36me3,Promoter_H3K4me1,Promoter_H3K4me2,Promoter_H3K4me3,Promoter_H3K79me2,Promoter_H3K9a,Promoter_H3K9me3,Promoter_H4K20me1,gene_name
geneId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2.0,2.0,0.0,1.0,2.0,0.0,A1BG
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,A2M
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A2MP1
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,NAT1
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NAT2


In [20]:
rna_seq.head()

,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation,gene_id_clean,gene_name
0,10904,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10904,BLCAP
1,12954,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12954,NaN
2,12956,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12956,NaN
3,12958,12958,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12958,NaN
4,12960,12960,73.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12960,NaN


In [21]:
rna_seq = rna_seq[['gene_name', 'TPM']].dropna()
rna_seq.shape

(42026, 2)

In [22]:
data = pd.merge(feature_matrix,rna_seq,on='gene_name').set_index('gene_name')

In [23]:
X, y = data.iloc[:, :-1], data.iloc[:, -1]
X.fillna(0, inplace = True)

/var/folders/gd/yfplhppn3jx0g03j2lr_yn6m0000gn/T/ipykernel_46973/225978411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace = True)


In [24]:
y = np.log10(y+0.001)

In [25]:
model = DecisionTreeRegressor()

In [28]:
cross_val_score(model, X, y)

array([ 0.4912271 ,  0.28363869,  0.21742942,  0.33846877, -1.02011341])

In [29]:
model.fit(X, y)

DecisionTreeRegressor()

In [34]:
len(model.feature_names_in_)

77

In [31]:
len(model.feature_importances_)

77

In [42]:
features_df = pd.DataFrame([model.feature_names_in_, model.feature_importances_]).T
features_df.columns = ['feature', 'score']
features_df.sort_values(by = 'score', ascending = False).head(10)

,feature,score
74,Promoter_H3K9a,0.51982
73,Promoter_H3K79me2,0.094367
72,Promoter_H3K4me3,0.035603
70,Promoter_H3K4me1,0.028647
69,Promoter_H3K36me3,0.026318
66,Promoter_H2AFZ,0.0168
58,Intron_H3K36me3,0.016694
71,Promoter_H3K4me2,0.016072
3,3' UTR_H3K36me3,0.015268
62,Intron_H3K79me2,0.014401
